## Imports

In [21]:
import os
import numpy as np
from numpy.random import default_rng
import cv2
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from tqdm import tqdm

## Settings

In [22]:
dataset = '/home/drevital/obstacles_classification_datasets/src_base_dataset/augment'
obstacle_dataset = os.path.join(dataset, 'obstacle')
no_obstacle_dataset = os.path.join(dataset, 'no_obstacle')
in_aug_dataset = '/home/drevital/obstacles_classification_datasets/src_base_dataset/augment'
out_aug_dataset = '/home/drevital/obstacles_classification_datasets/src_base_dataset/augmented'
in_obstacle_augment_dataset = os.path.join(in_aug_dataset, 'obstacle')
in_no_obstacle_augment_dataset = os.path.join(in_aug_dataset, 'no_obstacle')
out_obstacle_augment_dataset = os.path.join(out_aug_dataset, 'obstacle')
out_no_obstacle_augment_dataset = os.path.join(out_aug_dataset, 'no_obstacle')
obstacle_fnames = os.listdir(obstacle_dataset)
no_obstacle_fnames = os.listdir(no_obstacle_dataset)
in_obstacle_fnames = os.listdir(in_obstacle_augment_dataset)
in_no_obstacle_fnames = os.listdir(in_no_obstacle_augment_dataset)
current_obs = len(obstacle_fnames)
current_no_obs = len(no_obstacle_fnames)
current_in_obs = len(in_obstacle_fnames)
current_in_no_obs = len(in_no_obstacle_fnames)
target = 40
augments_obs = target - current_obs
augments_no_obs = target - current_no_obs

In [23]:
current_obs, current_no_obs, target, augments_obs, augments_no_obs

(11, 2, 40, 29, 38)

## Augmentation Functions

In [24]:
def rotate(i, im, im1, im2):
    degrees = np.random.randint(-20, 21)
    rotate = iaa.Affine(rotate=(degrees, degrees))
    rotated_im1 = rotate.augment_image(im1)
    rotated_im2 = rotate.augment_image(im2)
    rotated_image = cv2.hconcat([rotated_im1, rotated_im2])

    return rotated_image

def noise(i, im, im1, im2):
    gaussian_noise = iaa.AdditiveGaussianNoise(10, 20)
    noise_image = gaussian_noise.augment_image(im)
        
    return noise_image

def crop(i, im, im1, im2):
    percents = np.random.randint(0, 5)
    percent = percents / 100.0
    crop = iaa.Crop(percent=(percent, percent))
    crop_im1 = crop.augment_image(im1)
    crop_im2 = crop.augment_image(im2)
    crop_image = cv2.hconcat([crop_im1, crop_im2])

    return crop_image

def shear(i, im, im1, im2):
    shear_param = np.random.randint(0, 21)
    shear = iaa.Affine(shear=(shear_param, shear_param))
    shear_im1 = shear.augment_image(im1)
    shear_im2 = shear.augment_image(im2)
    shear_image = cv2.hconcat([shear_im1, shear_im2])

    return shear_image

def contrast(i, im, im1, im2):
    gammas = np.random.randint(70, 171)
    gamma = gammas / 100.0
    contrast = iaa.GammaContrast(gamma=gamma)
    contrast_im1 = contrast.augment_image(im1)
    contrast_im2 = contrast.augment_image(im2)
    contrast_image = cv2.hconcat([contrast_im1, contrast_im2])

    return contrast_image

def vertical_flip(i, im, im1, im2):
    flip_vr = iaa.Flipud(p=1.0)
    flip_vr_image = flip_vr.augment_image(im)

    return flip_vr_image

## Augmentation Handling Function

In [25]:
def augment(in_aug_dataset,
            out_aug_dataset,
            aug_dir,
            fnames,
            num_augments,
            rands,
            aug_names,
            display=False):
    
    for i, rand in tqdm(enumerate(rands)):    
        in_fname = fnames[rand]
        in_imname = in_fname.split('.')[0]
        in_impath = os.path.join(in_aug_dataset, aug_dir, in_fname)
        im = imageio.imread(in_impath)
        w = im.shape[1]
        im1 = im[:, :w//2]
        im2 = im[:, w//2:] 
        naugment = i % num_augments
        augmented = augment_fn[naugment](i, im, im1, im2)
        if display:
            ia.imshow(augmented)
        out_imname = in_imname + aug_names[naugment] + str(i)
        out_impath = os.path.join(out_aug_dataset, aug_dir, out_imname + '.jpg')
        imageio.imwrite(out_impath, augmented)

## Augment

In [26]:
rng = default_rng()
rand_obs = rng.choice(current_in_obs, size=augments_obs, replace=True)
rand_no_obs = rng.choice(current_in_no_obs, size=augments_no_obs, replace=True)
augment_fn = [rotate, crop, shear, contrast, vertical_flip]
num_augments = len(augment_fn)
aug_names = ['_rotate_',
             '_crop_',
             '_shear_',
             '_contrast_',
             '_vr_flip_']

augment(in_aug_dataset,
        out_aug_dataset,
        'obstacle',
        in_obstacle_fnames,
        num_augments,
        rand_obs,
        aug_names)        

augment(in_aug_dataset,
        out_aug_dataset,
        'no_obstacle',
        in_no_obstacle_fnames,
        num_augments,
        rand_no_obs,
        aug_names)

29it [00:00, 268.77it/s]
38it [00:00, 278.35it/s]
